## 1. Importations et Configuration Initiale

In [1]:
import json
import pandas as pd
import re
import torch.serialization
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
from datetime import datetime
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import joblib
from sklearn.preprocessing import MultiLabelBinarizer
import logging
from typing import List, Union
import json
import os
import warnings
from pathlib import Path

# Configuration du logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## 2. Classes et Fonctions Utilitaires


In [2]:
# Encoder personnalisé pour les types numpy
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.int32, np.int64)):
            return int(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

def convert_numpy_types(data):
    """Convertit récursivement les types numpy en types Python natifs"""
    if isinstance(data, dict):
        return {k: convert_numpy_types(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_numpy_types(item) for item in data]
    elif isinstance(data, (np.float32, np.float64)):
        return float(data)
    elif isinstance(data, (np.int32, np.int64)):
        return int(data)
    elif isinstance(data, np.ndarray):
        return data.tolist()
    return data


## 3. Connexion à MongoDB

In [3]:
def connect_mongodb():
    try:
        client = MongoClient(
            "mongodb://localhost:27017/",
            serverSelectionTimeoutMS=2000,
            connectTimeoutMS=10000,
            socketTimeoutMS=10000
        )
        client.admin.command('ping')
        db = client["PFE"]
        return db["resumes"]
    except Exception as e:
        logger.error(f"ERREUR MongoDB: {str(e)}")
        raise RuntimeError("Impossible de se connecter à MongoDB") from e

# Initialisation MongoDB
try:
    cvs_collection = connect_mongodb()
    cvs_data = list(cvs_collection.find({}))
    logger.info(f"{len(cvs_data)} CVs chargés depuis MongoDB")
except Exception as e:
    logger.error(f"Erreur d'initialisation: {e}")
    cvs_data = []

INFO:__main__:44 CVs chargés depuis MongoDB


## 4. Prédiction de Catégorie avec BERT

In [4]:
class CVCategoryPredictor:
    def __init__(self, model_path: str = None):
        """Initialise le prédicteur de catégories de CV.

        Args:
            model_path: Chemin vers le modèle pré-entraîné (dossier contenant les fichiers)
                      Si None, utilise le chemin par défaut dans le dossier models/
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.mlb = None
        self.tokenizer = None
        self.threshold = 0.3

        # Chemins absolus par défaut
        BASE_MODEL_PATH = r"C:\Users\AzComputer\PyCharmMiscProject\models"
        self.default_model_path = os.path.join(BASE_MODEL_PATH, "mon_modele_bert_multilabel")
        self.default_binarizer_path = os.path.join(BASE_MODEL_PATH, "multilabel_binarizer.pkl")

        # Liste des chemins possibles à essayer
        possible_paths = [
            self.default_model_path,  # Chemin absolu principal
            os.path.join(os.getcwd(), "models", "mon_modele_bert_multilabel"),  # Relatif au répertoire courant
            os.path.join("..", "models", "mon_modele_bert_multilabel")  # Un niveau au-dessus
        ]

        if model_path is not None:
            possible_paths.insert(0, model_path)  # Ajouter le chemin spécifié en premier

        # Trouver le premier chemin valide
        self.model_path = None
        for path in possible_paths:
            if os.path.exists(path):
                self.model_path = path
                break

        if self.model_path is None:
            logger.error(f"Aucun modèle trouvé aux emplacements:\n" + "\n".join(possible_paths))
            self._load_fallback_model()
            return

        try:
            # 1. Charger le binariseur
            binarizer_path = os.path.join(os.path.dirname(self.model_path), "multilabel_binarizer.pkl")
            self._load_label_binarizer(binarizer_path)

            # 2. Charger le modèle principal
            logger.info(f"Chargement du modèle depuis {self.model_path}")

            self.config = AutoConfig.from_pretrained(self.model_path)
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            self.model = AutoModelForSequenceClassification.from_pretrained(
                self.model_path,
                config=self.config
            ).to(self.device)
            self.model.eval()

            logger.info("Modèle principal chargé avec succès")

        except Exception as e:
            logger.error(f"Erreur lors du chargement du modèle principal: {e}")
            self._load_fallback_model()

    def _load_label_binarizer(self, path: str):
        """Charge le binariseur de labels avec gestion robuste."""
        try:
            with open(path, 'rb') as f:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=UserWarning)
                    self.mlb = joblib.load(f)

            if not hasattr(self.mlb, 'classes_') or len(self.mlb.classes_) == 0:
                raise ValueError("Binarizer chargé mais sans classes définies")

            logger.info(f"Binarizer chargé. Classes disponibles: {self.mlb.classes_}")
        except Exception as e:
            logger.error(f"Erreur lors du chargement du binarizer: {e}")
            self._create_fallback_binarizer()
    def _create_fallback_binarizer(self):
        """Crée un binariseur de secours avec une classe UNKNOWN."""
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit([['UNKNOWN']])
        logger.warning("Binarizer de secours créé avec la classe UNKNOWN")

    def _load_fallback_model(self):
        """Charge un modèle de repli en cas d'échec."""
        try:
            logger.warning("Chargement du modèle de repli (bert-base-uncased)")

            # Charger le tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

            # Configuration de base pour le modèle de repli
            self.config = AutoConfig.from_pretrained('bert-base-uncased')
            self.config.num_labels = 1  # Une seule classe pour UNKNOWN

            # Charger le modèle avec from_config pour éviter les poids non initialisés
            self.model = AutoModelForSequenceClassification.from_config(self.config).to(self.device)
            self.model.eval()

            if self.mlb is None:
                self._create_fallback_binarizer()

            logger.info("Modèle de repli chargé avec succès")

        except Exception as e:
            logger.error(f"Échec critique du chargement du modèle de repli: {e}")
            raise RuntimeError("Impossible de charger un modèle de repli")

    def predict_category(self, cv_data: dict) -> List[str]:
        """Prédit les catégories d'un CV.

        Args:
            cv_data: Données du CV sous forme de dictionnaire

        Returns:
            Liste des catégories prédites (ou ['UNKNOWN'] en cas d'erreur)
        """
        if self.model is None or self.mlb is None:
            logger.warning("Modèle non disponible - retour par défaut")
            return ["UNKNOWN"]

        try:
            # 1. Convertir le CV en texte
            text = self._cv_to_text(cv_data)
            if not text.strip():
                return ["UNKNOWN"]

            logger.debug(f"Texte généré pour la prédiction:\n{text[:500]}...")

            # 2. Tokenization (simplifiée pour le modèle de repli)
            inputs = self.tokenizer(
                text,
                return_tensors='pt',
                truncation=True,
                padding=True,
                max_length=512,
                return_overflowing_tokens=False  # Retiré car cause des problèmes avec BERT
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # 3. Prédiction
            with torch.no_grad():
                outputs = self.model(**inputs)

            # 4. Post-traitement
            logits = outputs.logits
            probs = torch.sigmoid(logits)
            logger.debug(f"Probabilités brutes: {probs.cpu().numpy()}")

            # Appliquer le seuil
            predictions = (probs > self.threshold).int().cpu().numpy()

            # Convertir en labels
            predicted_labels = self.mlb.inverse_transform(predictions)
            result = list(predicted_labels[0]) if len(predicted_labels[0]) > 0 else ["UNKNOWN"]

            logger.info(f"Catégories prédites: {result}")
            return result

        except Exception as e:
            logger.error(f"Erreur lors de la prédiction: {str(e)}", exc_info=True)
            return ["UNKNOWN"]

    def _cv_to_text(self, cv_data: dict) -> str:
        """Convertit les données structurées du CV en texte pour l'analyse.

        Args:
            cv_data: Données du CV sous forme de dictionnaire

        Returns:
            Texte formaté pour l'analyse
        """
        sections = []

        # Informations de base
        if cv_data.get('name'):
            sections.append(f"CANDIDAT: {cv_data['name']}")

        # Compétences
        if cv_data.get('skills'):
            skills = [str(s).strip() for s in cv_data['skills'] if s and str(s).strip()]
            if skills:
                sections.append("COMPÉTENCES: " + ', '.join(skills))

        # Expériences professionnelles
        experiences = cv_data.get('experiences', [])
        exp_texts = []
        for exp in experiences:
            parts = []
            if exp.get('job_title'):
                parts.append(f"Poste: {exp['job_title']}")
            if exp.get('company'):
                parts.append(f"Entreprise: {exp['company']}")
            if exp.get('duration'):
                parts.append(f"Durée: {exp['duration']}")
            if exp.get('description'):
                desc = str(exp['description']).replace('\n', ' ').strip()
                parts.append(f"Description: {desc}")
            if parts:
                exp_texts.append(' | '.join(parts))
        if exp_texts:
            sections.append("EXPÉRIENCES PROFESSIONNELLES: " + ' || '.join(exp_texts))

        # Formation
        education = cv_data.get('education', [])
        edu_texts = []
        for edu in education:
            parts = []
            if edu.get('degree'):
                parts.append(f"Diplôme: {edu['degree']}")
            if edu.get('institution'):
                parts.append(f"Établissement: {edu['institution']}")
            if edu.get('year'):
                parts.append(f"Année: {edu['year']}")
            if parts:
                edu_texts.append(' - '.join(parts))
        if edu_texts:
            sections.append("FORMATION: " + ' | '.join(edu_texts))

        # Projets
        projects = cv_data.get('projects', [])
        if projects:
            proj_texts = []
            for proj in projects:
                parts = []
                if proj.get('project_title'):
                    parts.append(f"Titre: {proj['project_title']}")
                if proj.get('project_summary'):
                    summary = str(proj['project_summary']).replace('\n', ' ').strip()
                    parts.append(f"Résumé: {summary}")
                if parts:
                    proj_texts.append(' | '.join(parts))
            if proj_texts:
                sections.append("PROJETS: " + ' || '.join(proj_texts))

        # Certifications
        if cv_data.get('certifications'):
            certs = [str(c).strip() for c in cv_data['certifications'] if c and str(c).strip()]
            if certs:
                sections.append("CERTIFICATIONS: " + ', '.join(certs))

        return '\n\n'.join(sections).strip()

    def set_threshold(self, threshold: float):
        """Définit le seuil de probabilité pour les prédictions.

        Args:
            threshold: Seuil entre 0 et 1
        """
        if 0 <= threshold <= 1:
            self.threshold = threshold
            logger.info(f"Nouveau seuil de prédiction: {threshold}")
        else:
            logger.warning(f"Seuil invalide: {threshold}. Doit être entre 0 et 1.")

## 5. Embedding de Compétences avec BERT

In [5]:
class SkillEmbedder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased')
        self.model.eval()
        self.cache = {}

    def embed_skill(self, skill_text):
        if isinstance(skill_text, dict):
            skill_text = str(skill_text)

        if skill_text in self.cache:
            return self.cache[skill_text]
        try:
            inputs = self.tokenizer(skill_text, return_tensors='pt', truncation=True, padding=True)
            with torch.no_grad():
                outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            self.cache[skill_text] = embedding
            return embedding
        except:
            return np.zeros(768)


## 6. Company Classifier Implementation

In [6]:
COMPANY_WEIGHTS = {
    'large_company': 1.5,  # Augmenté de 1.2 à 1.5
    'startup': 0.7,       # Réduit de 0.8 à 0.7
    'default': 1.0
}

class CompanyClassifier(nn.Module):
    """Modèle complet de classification d'entreprises avec mécanisme d'attention"""

    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, num_layers=1, dropout=0.5):
        super().__init__()

        # Couche d'embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(dropout)

        # Couche LSTM bidirectionnelle
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers>1 else 0
        )

        # Mécanisme d'attention
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim * 2),
            nn.Tanh(),
            nn.Linear(hidden_dim * 2, 1)
        )

        # Classificateur
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # Embedding des tokens
        embedded = self.embedding_dropout(self.embedding(x))

        # Passage dans le LSTM
        lstm_out, _ = self.lstm(embedded)

        # Application de l'attention
        attention_weights = F.softmax(self.attention(lstm_out), dim=1)
        attn_out = torch.sum(attention_weights * lstm_out, dim=1)

        # Classification finale
        out = self.dropout(F.relu(self.fc1(attn_out)))
        return torch.sigmoid(self.fc2(out)).squeeze(-1), attention_weights.squeeze(-1)

# Définir la fonction tokenizer_func au niveau global
def tokenizer_func(text):
    """Tokeniseur personnalisé pour les noms d'entreprises"""
    return re.findall(r'\w+', text.lower())

torch.serialization.add_safe_globals([tokenizer_func])

class CompanyClassifierWrapper:
    def __init__(self, model_path=None):
        # Chemin réel du modèle
        actual_model_path = r"C:\Users\AzComputer\PyCharmMiscProject\models\company_classifier.pth"

        # Déterminer le chemin à utiliser
        if model_path is None:
            model_path = actual_model_path
        else:
            # Si un chemin personnalisé est fourni, vérifier qu'il existe
            if not os.path.exists(model_path):
                logger.warning(f"Le chemin spécifié {model_path} n'existe pas, utilisation du chemin par défaut")
                model_path = actual_model_path

        logger.info(f"Chargement du modèle depuis: {model_path}")

        try:
            if not os.path.exists(model_path):
                raise FileNotFoundError(f"Fichier modèle introuvable: {model_path}. Veuillez vérifier que:"
                                     f"\n1. Le fichier existe à cet emplacement"
                                     f"\n2. Le chemin est correct"
                                     f"\n3. Vous avez les permissions de lecture")

            # Charger le modèle pré-entraîné
            saved_data = torch.load(model_path, map_location='cpu', weights_only=False)

            # Ajuster les clés du state_dict si nécessaire
            state_dict = saved_data['model_state_dict']
            adjusted_state_dict = {}

            for key, value in state_dict.items():
                # Corriger les clés du mécanisme d'attention
                if key.startswith('attention.attention.'):
                    new_key = key.replace('attention.attention.', 'attention.')
                    adjusted_state_dict[new_key] = value
                    logger.debug(f"Clé ajustée: {key} -> {new_key}")
                else:
                    adjusted_state_dict[key] = value

            self.model = CompanyClassifier(
                vocab_size=len(saved_data['vocab']),
                embedding_dim=128,
                hidden_dim=128
            )

            # Charger le state_dict ajusté
            load_result = self.model.load_state_dict(adjusted_state_dict, strict=False)
            logger.info(f"Résultat du chargement du modèle: {load_result}")

            self.model.eval()
            self.vocab = saved_data['vocab']
            self.max_length = saved_data['max_length']
            self.tokenizer_func = tokenizer_func

            logger.info("CompanyClassifier chargé avec succès")

        except Exception as e:
            logger.error(f"Erreur lors du chargement du CompanyClassifier: {e}")
            self.model = None

    def predict_company_size(self, company_name):
        if self.model is None:
            logger.warning(f"Modèle non chargé, utilisation du poids par défaut pour: {company_name}")
            return COMPANY_WEIGHTS['default']

        try:
            # Tokenisation et conversion numérique
            tokens = self.tokenizer_func(company_name)
            numericalized = [self.vocab.get(token, self.vocab['<unk>']) for token in tokens]

            if len(numericalized) > self.max_length:
                numericalized = numericalized[:self.max_length]
            else:
                numericalized = numericalized + [self.vocab['<pad>']] * (self.max_length - len(numericalized))

            # Conversion en tensor
            input_tensor = torch.tensor([numericalized], dtype=torch.long)

            # Prédiction
            with torch.no_grad():
                output, _ = self.model(input_tensor)
                prob = output.item()

            # Détermination du type d'entreprise
            if prob > 0.7:  # Seuil élevé pour startup
                company_type = 'startup'
                weight = COMPANY_WEIGHTS['startup']
            elif prob < 0.3:  # Seuil bas pour grande entreprise
                company_type = 'large_company'
                weight = COMPANY_WEIGHTS['large_company']
            else:
                company_type = 'default'
                weight = COMPANY_WEIGHTS['default']

            logger.info(f"Entreprise: {company_name} | Type prédit: {company_type} (prob: {prob:.2f}) | Poids appliqué: {weight}")
            return weight

        except Exception as e:
            logger.error(f"Erreur lors de la prédiction pour {company_name}: {e}")
            return COMPANY_WEIGHTS['default']

# Initialiser le wrapper avec logging détaillé
logger.info("Initialisation du CompanyClassifierWrapper...")
company_classifier = CompanyClassifierWrapper()
logger.info("CompanyClassifierWrapper initialisé")

INFO:__main__:Initialisation du CompanyClassifierWrapper...
INFO:__main__:Chargement du modèle depuis: C:\Users\AzComputer\PyCharmMiscProject\models\company_classifier.pth
INFO:__main__:Résultat du chargement du modèle: <All keys matched successfully>
INFO:__main__:CompanyClassifier chargé avec succès
INFO:__main__:CompanyClassifierWrapper initialisé


## 7. Calcul des Scores de Compétences

In [7]:

def calculate_skill_scores(cvs_data):
    """Version optimisée et robuste qui gère tous les cas de valeurs None"""
    from collections import defaultdict
    import numpy as np

    # 1. Initialisation et comptage des fréquences
    global_skill_freq = defaultdict(int)
    company_weights_cache = {}

    # Compter les occurrences globales des compétences
    for cv in cvs_data:
        skills = cv.get('skills', []) or []  # Gestion du cas None
        for skill in skills:
            skill_name = skill.get('name', str(skill)) if isinstance(skill, dict) else str(skill)
            global_skill_freq[skill_name] += 1

    # 2. Nouvelle structure de données enrichie avec mentions GitHub
    skill_info = defaultdict(lambda: {
        'exp_mentions': 0,
        'proj_mentions': 0,
        'edu_mentions': 0,
        'cert_mentions': 0,
        'github_mentions': 0,  # Nouveau champ pour les mentions GitHub
        'years_sum': 0,
        'count': 0,
        'company_weights': [],
        'prestigious_companies': 0
    })

    # 3. Collecte complète des données avec gestion robuste des None
    for cv in cvs_data:
        skills = cv.get('skills', []) or []
        years_exp = cv.get('years_of_experience', 0) or 0

        # Pré-calcul des poids d'entreprise pour ce CV
        exp_company_weights = []
        for exp in cv.get('experiences', []) or []:
            company_name = (exp.get('company', '') or '').strip()
            if company_name:
                if company_name not in company_weights_cache:
                    weight = company_classifier.predict_company_size(company_name)
                    company_weights_cache[company_name] = weight
                    if weight > 1.0:
                        skill_info['__prestigious_total'] = skill_info.get('__prestigious_total', 0) + 1
                exp_company_weights.append(company_weights_cache[company_name])
            else:
                exp_company_weights.append(1.0)

        # Analyse détaillée par compétence
        for skill in skills:
            skill_name = skill.get('name', str(skill)) if isinstance(skill, dict) else str(skill)
            info = skill_info[skill_name]
            info['count'] += 1
            info['years_sum'] += years_exp

            # Vérification des mentions dans les expériences
            for i, exp in enumerate(cv.get('experiences', []) or []):
                desc = (exp.get('description', '') or '').lower()
                if skill_name.lower() in desc:
                    info['exp_mentions'] += 1
                    if i < len(exp_company_weights):
                        weight = exp_company_weights[i]
                        info['company_weights'].append(weight)
                        if weight > 1.0:
                            info['prestigious_companies'] += 1

            # Vérification des mentions dans les projets
            for proj in cv.get('projects', []) or []:
                project_summary = (proj.get('project_summary', '') or '').lower()
                if skill_name.lower() in project_summary:
                    info['proj_mentions'] += 1

            # Vérification des mentions dans l'éducation
            for edu in cv.get('education', []) or []:
                degree = (edu.get('degree', '') or '').lower()
                institution = (edu.get('institution', '') or '').lower()
                if skill_name.lower() in degree or skill_name.lower() in institution:
                    info['edu_mentions'] += 1

            # Vérification des mentions dans les certifications
            for cert in cv.get('certifications', []) or []:
                cert_str = (cert if isinstance(cert, str) else str(cert)) or ''
                if skill_name.lower() in cert_str.lower():
                    info['cert_mentions'] += 1

            # Vérification des mentions dans les projets GitHub (nouveau)
            for gh_proj in cv.get('github_projects', []) or []:
                # Vérification dans la description
                desc = (gh_proj.get('description', '') or '').lower()
                if skill_name.lower() in desc:
                    info['github_mentions'] += 1

                # Vérification dans les langages utilisés
                for lang in gh_proj.get('languages', []) or []:
                    if skill_name.lower() == lang.lower():
                        info['github_mentions'] += 1

    # 4. Calcul des scores optimisés
    total_prestigious = max(skill_info.get('__prestigious_total', 1), 1)
    skill_scores = {}

    for skill_name, info in skill_info.items():
        if skill_name == '__prestigious_total':
            continue

        total = max(info['count'], 1)

        # Calcul du company_score avec boost
        if info['company_weights']:
            max_weight = max(info['company_weights'])
            company_score = 1 + (max_weight - 1) * 1.2
        else:
            company_score = 1.0

        # Bonus de prestige
        prestige_bonus = min(info['prestigious_companies'] / total_prestigious * 0.15, 0.15)

        # Formule principale avec ajout des mentions GitHub (poids faible: 0.05)
        base_score = (
            0.45 * (info['exp_mentions'] / total) * company_score +
            0.3 * (info['proj_mentions'] / total) +
            0.15 * (info['edu_mentions'] / total) +
            0.1 * (info['cert_mentions'] / total) +
            0.05 * (info['github_mentions'] / total) +  # Nouveau terme GitHub
            0.05 * min(info['years_sum'] / (total * 5), 1) +
            prestige_bonus
        )

        # Ajustement de fréquence
        freq_adj = 1 / (1 + np.log(1 + global_skill_freq[skill_name] / len(cvs_data)))**0.3
        adj_score = min(base_score * freq_adj * 1.1, 0.99)

        skill_scores[skill_name] = adj_score

    # 5. Normalisation intelligente
    if skill_scores:
        scores = np.array(list(skill_scores.values()))
        p25, p75 = np.percentile(scores, [25, 75])
        iqr = max(p75 - p25, 1e-8)

        for skill in skill_scores:
            if iqr > 0:
                normalized = 0.4 + 0.5 * ((skill_scores[skill] - p25) / iqr)
            else:
                normalized = 0.6

            skill_scores[skill] = min(max(normalized, 0.3), 0.95)

    return skill_scores

## 8. Calcul de Similarité entre Compétences

In [8]:
def compute_skill_similarity(skill_scores):
    skills = list(skill_scores.keys())
    embeddings = []

    for skill in skills:
        try:
            embeddings.append(skill_embedder.embed_skill(skill))
        except:
            embeddings.append(np.zeros(768))

    similarity_matrix = cosine_similarity(np.array(embeddings))

    # Convertir en types Python natifs pour MongoDB
    similarity_matrix = convert_numpy_types(similarity_matrix)

    return skills, similarity_matrix


## 9. Calcul des Scores pour les CVs

In [9]:
def calculate_cv_scores(cv_data, skill_scores, skill_similarity):
    """Version optimisée avec final_score = average_skill_score"""
    skills, similarity_matrix = skill_similarity
    cv_skills = cv_data.get('skills', [])
    skill_scores_in_cv = []
    max_possible_score = 0.95  # Plafond global conservé

    # 1. Calcul des scores individuels avec boost (inchangé)
    for skill in cv_skills:
        skill_str = skill.get('name', str(skill)) if isinstance(skill, dict) else str(skill)

        if skill_str in skill_scores:
            base_score = skill_scores[skill_str]

            # Boost de similarité
            try:
                idx = skills.index(skill_str)
                similar_indices = [i for i in range(len(skills)) if similarity_matrix[idx][i] > 0.4]

                if similar_indices:
                    similar_scores = []
                    for i in similar_indices:
                        sim = similarity_matrix[idx][i]
                        weight = sim ** 1.5
                        similar_scores.append((skill_scores[skills[i]], weight))

                    total_weight = sum(w for _, w in similar_scores)
                    if total_weight > 0:
                        weighted_avg = sum(s * w for s, w in similar_scores) / total_weight
                        boosted_score = min(base_score * 0.6 + weighted_avg * 0.4, max_possible_score)
                    else:
                        boosted_score = base_score
                else:
                    boosted_score = base_score
            except:
                boosted_score = base_score

            # Bonus pour compétences principales
            if boosted_score > 0.7:
                boosted_score = min(boosted_score * 1.05, max_possible_score)

            skill_scores_in_cv.append(boosted_score)

    # 2. Calcul de la moyenne pondérée (conservée mais résultat utilisé différemment)
    if skill_scores_in_cv:
        weights = [s**3 for s in skill_scores_in_cv]
        avg_skill_score = np.average(skill_scores_in_cv, weights=weights)
    else:
        avg_skill_score = 0

    # 3. Formattage des résultats avec final_score = average_skill_score
    return {
        'skill_scores': {s.get('name', str(s)) if isinstance(s, dict) else str(s): float(score)
                        for s, score in zip(cv_skills, skill_scores_in_cv)},
        'average_skill_score': float(avg_skill_score),
        'final_score': float(avg_skill_score)  # Ici on égalise explicitement les deux valeurs
    }

## 10. Similarité entre CVs par Catégorie

In [10]:
def calculate_cv_similarity_by_category(cvs_collection):
    """Calcule la similarité entre CVs de la même catégorie en utilisant seulement les CVs de référence"""
    # Récupérer toutes les catégories distinctes (en considérant que chaque CV peut avoir plusieurs catégories)
    all_categories = set()
    for cv in cvs_collection.find({}, {"categories": 1}):
        all_categories.update(cv.get("categories", []))

    category_results = {}

    for category in all_categories:
        # Récupérer tous les CVs qui ont cette catégorie avec leurs scores
        cvs_in_category = list(cvs_collection.find({"categories": category},
                                                  {"name": 1, "final_score": 1, "_id": 1, "categories": 1}))

        if len(cvs_in_category) < 2:
            logger.info(f"Catégorie {category} n'a qu'un seul CV, pas de similarité à calculer")
            continue

        # Trier les CVs par score
        cvs_sorted = sorted(cvs_in_category, key=lambda x: x.get('final_score', 0))

        # Sélectionner les CVs de référence
        reference_cvs = [
            cvs_sorted[0],  # Score min
            cvs_sorted[-1],  # Score max
        ]

        # Ajouter deux CVs médians si possible
        if len(cvs_sorted) > 2:
            mid = len(cvs_sorted) // 2
            reference_cvs.append(cvs_sorted[mid])
            if len(cvs_sorted) > 3:
                reference_cvs.append(cvs_sorted[mid-1] if mid > 1 else cvs_sorted[1])

        # Charger les données complètes des CVs de référence
        reference_cvs_full = list(cvs_collection.find(
            {"_id": {"$in": [cv["_id"] for cv in reference_cvs]}}
        ))

        # Préparer les embeddings pour chaque CV de référence
        cv_texts = []
        cv_embeddings = []
        reference_names = []

        for cv in reference_cvs_full:
            try:
                # Convertir le CV en texte
                text = category_predictor._cv_to_text(cv)
                cv_texts.append(text)
                reference_names.append(cv.get('name', 'Inconnu'))

                # Générer l'embedding avec BERT
                inputs = skill_embedder.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
                with torch.no_grad():
                    outputs = skill_embedder.model(**inputs)
                embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
                cv_embeddings.append(embedding)
            except Exception as e:
                logger.error(f"Erreur lors du traitement du CV {cv.get('name', 'Inconnu')}: {e}")
                continue

        if len(cv_embeddings) < 2:
            logger.info(f"Catégorie {category} n'a pas assez de CVs de référence valides pour calculer la similarité")
            continue

        # Calculer la matrice de similarité entre les CVs de référence
        similarity_matrix = cosine_similarity(np.array(cv_embeddings))

        # Stocker les résultats
        category_results[category] = {
            'reference_cvs': reference_names,
            'reference_scores': [cv.get('final_score', 0) for cv in reference_cvs_full],
            'similarity_matrix': convert_numpy_types(similarity_matrix),
            'average_similarity': float(np.mean(similarity_matrix))
        }

        logger.info(f"Similarité calculée pour la catégorie {category} avec {len(reference_cvs_full)} CVs de référence, similarité moyenne: {category_results[category]['average_similarity']:.2f}")

    return category_results

## 11. Mise à Jour des CVs dans MongoDB

In [11]:
def update_resumes_in_mongodb(cvs_collection, cvs_data, skill_scores, skill_similarity):
    category_predictor = CVCategoryPredictor()
    update_results = []

    for cv in cvs_data:
        try:
            # Calcul des scores pour ce CV
            scores = calculate_cv_scores(cv, skill_scores, skill_similarity)

            # Conversion des types numpy
            scores = convert_numpy_types(scores)

            # Prédiction de la catégorie (peut retourner plusieurs catégories maintenant)
            categories = category_predictor.predict_category(cv)

            # Préparation des compétences avec scores
            skills_with_scores = []
            existing_skills = cv.get('skills', [])

            for skill in existing_skills:
                skill_name = skill.get('name', str(skill)) if isinstance(skill, dict) else str(skill)

                # Création du nouvel objet skill avec le score
                updated_skill = {
                    'name': skill_name,
                    'score': scores['skill_scores'].get(skill_name, 0.0)
                }

                # Si la compétence originale était un dictionnaire, conserver tous les champs existants
                if isinstance(skill, dict):
                    updated_skill.update({k: v for k, v in skill.items() if k != 'score'})

                skills_with_scores.append(updated_skill)

            # Préparation de la mise à jour
            update_data = {
                'categories': categories,  # Maintenant une liste de catégories
                'skills': skills_with_scores,
                'average_skill_score': scores['average_skill_score'],
                'final_score': scores['final_score'],
                'last_updated': datetime.now()
            }

            # Mise à jour dans MongoDB
            result = cvs_collection.update_one(
                {'_id': cv['_id']},
                {'$set': update_data},
                upsert=False
            )

            if result.modified_count == 1:
                logger.info(f"CV {cv.get('name', 'Inconnu')} mis à jour avec succès")
            else:
                logger.warning(f"Aucune modification pour le CV {cv.get('name', 'Inconnu')}")

            update_results.append(result)

        except Exception as e:
            logger.error(f"Erreur lors de la mise à jour du CV {cv.get('name', 'Inconnu')}: {str(e)}")

    return update_results

## 12. Exécution Principale

In [12]:
# Initialisation des composants
skill_embedder = SkillEmbedder()
category_predictor = CVCategoryPredictor()

# Calcul des scores de compétences
skill_scores = calculate_skill_scores(cvs_data)
skill_scores = convert_numpy_types(skill_scores)

# Calcul de la similarité entre compétences
skill_similarity = compute_skill_similarity(skill_scores)

# Mise à jour des CVs dans MongoDB
update_results = update_resumes_in_mongodb(cvs_collection, cvs_data, skill_scores, skill_similarity)

# Calcul de la similarité entre CVs par catégorie
cv_similarity_by_category = calculate_cv_similarity_by_category(cvs_collection)


INFO:__main__:Binarizer chargé. Classes disponibles: ['ACCOUNTANT' 'ADVOCATE' 'AGRICULTURE' 'AI Specialist' 'API Developer'
 'APPAREL' 'ARTS' 'AUTOMOBILE' 'AVIATION' 'Advocate'
 'Analytics Professional' 'Angular Developer' 'Arts' 'Automation Testing'
 'BANKING' 'BPO' 'BUSINESS-DEVELOPMENT' 'Backend Developer'
 'Big Data Engineer' 'Blockchain' 'Blockchain Developer'
 'Business Analyst' 'CHEF' 'CI/CD Specialist' 'CONSTRUCTION' 'CONSULTANT'
 'Civil Engineer' 'Cloud Engineer' 'Cloud Practitioner'
 'Computer Vision Specialist' 'Containerization Expert' 'DESIGNER'
 'DIGITAL-MEDIA' 'Data Science' 'Data Scientist' 'Database'
 'Deep Learning Engineer' 'DevOps Engineer' 'Django Developer'
 'DotNet Developer' 'ENGINEERING' 'ETL Developer' 'Electrical Engineering'
 'FINANCE' 'FITNESS' 'Flask Developer' 'Frontend Developer'
 'Full Stack Developer' 'HEALTHCARE' 'HR' 'Hadoop' 'Health and fitness'
 'INFORMATION-TECHNOLOGY' 'IoT Developer' 'Java Developer'
 'JavaScript Developer' 'ML Engineer' 'Machine

## 13. Résultats et Export

In [13]:
# Statistiques des mises à jour
success_count = sum(1 for r in update_results if r is not None and r.matched_count > 0)
logger.info(f"Mise à jour terminée: {success_count}/{len(cvs_data)} CVs mis à jour avec succès")

# Export des résultats
try:
    # Export JSON
    with open('../cv_scores_results.json', 'w', encoding='utf-8') as f:
        json.dump({
            'skill_scores': skill_scores,
            'cv_similarity_by_category': cv_similarity_by_category,
            'update_count': success_count,
            'timestamp': datetime.now().isoformat()
        }, f, indent=2, cls=NumpyEncoder)

    # Export CSV des scores
    scores_df = pd.DataFrame([
        {
            'name': cv.get('name', 'Inconnu'),
            'category': cv.get('category', 'UNKNOWN'),
            'final_score': cv.get('final_score', 0),
            'skills_count': len(cv.get('skills', [])),
            'last_updated': cv.get('last_updated', datetime.now())
        }
        for cv in cvs_collection.find({}, {'name': 1, 'category': 1, 'final_score': 1, 'skills': 1, 'last_updated': 1})
    ])
    scores_df.to_csv('cv_scores_results.csv', index=False, encoding='utf-8')

    logger.info("Export des résultats terminé avec succès")
except Exception as e:
    logger.error(f"Erreur lors de l'export des résultats: {e}")

INFO:__main__:Mise à jour terminée: 44/44 CVs mis à jour avec succès
INFO:__main__:Export des résultats terminé avec succès
